In [1]:
library(tidyverse)
library(clusterProfiler)
library(WGCNA)
library(parallel)
library(pheatmap)
library(RColorBrewer)
library(ggplotify)
library(grid)

# Custom package
library(rutils)

-- Attaching packages ---------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



clusterProfiler v3.18.0  For help: https://guangchuangyu.github.io/software/clusterProfiler

If you use clusterProfiler in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Yanyan Han, Qing-Yu He. clusterProfiler: an R package for comparing biological themes among gene clusters. OMICS: A Journal of Integrative Biology. 2012, 16(5):284-287.


Attaching package: 'clusterProfiler'


The following object is masked from 'package:purrr':

    simplify


The following object is masked from 'package:stats':

    filter


Loading required pa

In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)

In [43]:
dset_idx <- 3

In [44]:
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))

In [45]:
# Subtract 1 from diagonal of TOM (so we don't count self-TOM)
tom_hm <- tom - diag(1, nrow(tom), nrow(tom))
tom_summary_df <- tibble(geneID = colnames(data_expr)) %>%
    mutate(tom_conn_sums = rowSums(tom_hm)) %>%
    mutate(tom_conn_means = rowMeans(tom_hm))
#     arrange(desc(tom_conn))
mean(tom_summary_df$tom_conn_sums)
quantile(tom_summary_df$tom_conn_sums)
mean(tom_summary_df$tom_conn_means)
quantile(tom_summary_df$tom_conn_means) %>% round(3)

[1] 3.942757

0%        25%        50%        75%       100% 
 0.3000765  1.7082870  2.6340958  4.4939226 26.5164136

[1] 0.00391535

0%   25%   50%   75%  100% 
0.000 0.002 0.003 0.004 0.026